In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout, LSTM
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error

In [ ]:
SPX = pd.read_fwf('/content/sample_data/SPX.txt' ,names=['Date', 'Close'], parse_dates=True)

In [ ]:
SPX.head()

In [ ]:
#SPX.Date = pd.to_datetime(SPX.Date, infer_datetime_format=True)

In [ ]:
VIX = pd.read_fwf('/content/sample_data/VIX.txt' ,names=['Date', 'Close'], parse_dates=True)

In [ ]:
VIX.head()

In [ ]:
SPX.info()

In [ ]:
SPX.describe()

In [ ]:
SPX['Close'] = SPX['Close'].apply(lambda x: float(x.split()[0].replace(',', '.')))
VIX['Close'] = VIX['Close'].apply(lambda x: float(x.split()[0].replace(',', '.')))

In [ ]:
# SPX = SPX.set_index('Date')
# VIX = VIX.set_index('Date')

In [ ]:
#SPX.index = pd.to_datetime(SPX.index,unit='ns')
#VIX.index = pd.to_datetime(VIX.index,unit='ns')
print(SPX.index)
print(VIX.index)

In [ ]:
SPX.head()

In [ ]:
SPX.info()

## **EVOLUTION DES NOS DONNEES**

In [ ]:
def Plotting_evolution(data):
    data['Close'].plot(figsize=(18, 6))
    plt.show()

In [ ]:
SPX_ = SPX.set_index('Date')
VIX_ = VIX.set_index('Date')
data = [SPX_, VIX_]
Names =  ['SPX_Data', 'VIX_Data']

for i in range(len(data)):
    print('EVOLUTION DE {} DANS LE TEMPS'.format(Names[i]))
    Plotting_evolution(data[i])

In [ ]:
SPX_data = SPX.copy()
SPX_data['Date'] = pd.to_datetime(SPX_data['Date'])
SPX_data = SPX_data.set_index('Date')
SPX_data.head()

In [ ]:
m = SPX_data['Close'].resample('W').agg(['mean', 'std', 'min', 'max'])

plt.figure(figsize=(12, 8))
m['mean']['2018'].plot(label='moyenne par semaine')
plt.fill_between(m.index, m['max'], m['min'], alpha=0.2, label='min-max par semaine')

plt.legend()
plt.show()

In [ ]:
SPX_data[['Close']].plot(subplots=True, figsize=(12, 8))

In [ ]:
#df = SPX.copy()
#df.groupby('Date').resample('D').mean()

## SPLIT DATA

In [ ]:
train_SPX = SPX.loc[0:1110,'Close']
test_SPX = SPX.loc[1110:,'Close']

def line_plot(line1, line2, label1=None, label2=None, title='', lw=2):
    fig, ax = plt.subplots(1, figsize=(14, 9))
    ax.plot(line1, label=label1, linewidth=lw)
    ax.plot(line2, label=label2, linewidth=lw)
    ax.set_ylabel('SPX/USDT', fontsize=14)
    ax.set_title(title, fontsize=16)
    ax.legend(loc='best', fontsize=16);

In [ ]:
line_plot(train_SPX, test_SPX, 'training', 'test', title="DONNEE D'ENTRAINEMNT ET DE TEST")

---

## ON VEUT PREDIRE LE COURS DES 7 PROCHAIN JOURS 

In [ ]:
# train_SPX = SPX.loc[0:1292,'Close']
# test_SPX = SPX.loc[1285:,'Close']
# 
# def line_plot(line1, line2, label1=None, label2=None, title='', lw=2):
#     fig, ax = plt.subplots(1, figsize=(14, 9))
#     ax.plot(line1, label=label1, linewidth=lw)
#     ax.plot(line2, label=label2, linewidth=lw)
#     ax.set_ylabel('SPX/USDT', fontsize=14)
#     ax.set_title(title, fontsize=16)
#     ax.legend(loc='best', fontsize=16);
#     
# line_plot(train_SPX, test_SPX, 'training', 'test', title='')

## TRAINING DATA

In [ ]:
train_SPX.plot(figsize=(16,8))

## TEST DATA

In [ ]:
test_SPX.plot(figsize=(16,8))

# NORMALISATION DE DONNEES


In [ ]:
def normalise_zero_base(data):
    return data / data.iloc[0] - 1

def normalise_min_max(data):
    return (data - data.min()) / (data.max() - data.min())

In [ ]:
def extract_window_data(data, window_len=5, zero_base=True):
    window_data = []
    for idx in range(len(data) - window_len):
        tmp = data[idx: (idx + window_len)].copy()
        if zero_base:
            tmp = normalise_zero_base(tmp)
        window_data.append(tmp.values)
    return np.array(window_data)

In [ ]:
def prepare_data(train_SPX, test_SPX,window_len=10, zero_base=True, test_size=0.2):
    X_train = extract_window_data(train_SPX, window_len, zero_base)
    X_test = extract_window_data(test_SPX, window_len, zero_base)
    y_train = train_SPX[window_len:].values
    y_test = test_SPX[window_len:].values
    if zero_base:
        y_train = y_train / train_SPX[:-window_len].values - 1
        y_test = y_test / test_SPX[:-window_len].values - 1
    X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
    X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

    return train_SPX, test_SPX, X_train, X_test, y_train, y_test

# MODELISATION 1 LSTM

- Creation de modele lstm

In [ ]:
def build_lstm_model(input_data, output_size, neurons, activ_func='linear',
                     dropout=0.2, loss='mse', optimizer='adam'):
    model = Sequential()
    model.add(LSTM(neurons, input_shape=(input_data.shape[1], input_data.shape[2])))
    model.add(Dropout(dropout))
    model.add(Dense(units=output_size))
    model.add(Activation(activ_func))

    model.compile(loss=loss, optimizer=optimizer)
    return model

- Les paramettres pour l'entrainement du model

In [ ]:
np.random.seed(245)
window_len = 5
test_size = 0.2
zero_base = True
lstm_neurons = 50
epochs = 40
batch_size = 32
loss = 'mse'
dropout = 0.24
optimizer = 'adam'

## SPLIT DATA

In [ ]:
train_data, test_data, X_train, X_test, y_train, y_test = prepare_data(
data, window_len=window_len, zero_base=zero_base, test_size=test_size)

In [ ]:
print('Len of X training data', X_train.shape)
print('Len of Y training data',y_train.shape)
print('------------------------------')
print('Len of X test data',X_test.shape)
print('Len of y test data',y_test.shape)

## TRAINING MODELE 1 LSTM

In [ ]:
model = build_lstm_model(
    X_train, output_size=1, neurons=lstm_neurons, dropout=dropout, loss=loss,
    optimizer=optimizer)
modelfit = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size, verbose=1, shuffle=True)
     

## COURBE D'ENTRAINEMENT ET DE TEST

In [ ]:
plt.plot(modelfit.history['loss'],'r',linewidth=2, label='Training loss')
plt.plot(modelfit.history['val_loss'], 'g',linewidth=2, label='Validation loss')
plt.title('LSTM Neural Networks - SPX Model')
plt.xlabel('Epochs numbers')
plt.ylabel('MSE numbers')
plt.show()

In [ ]:
targets = test_SPX[window_len:]
preds = model.predict(X_test).squeeze()
mean_absolute_error(preds, y_test)

In [ ]:
from sklearn.metrics import mean_squared_error
SCORE_MSE=mean_squared_error(preds, y_test)
SCORE_MSE

In [ ]:
from sklearn.metrics import r2_score
r2_score1=r2_score(y_test, preds)
r2_score1*100

In [ ]:
preds = test_SPX.values[:-window_len] * (preds + 1)
preds

In [ ]:
#preds.plot(figsize=(16,8))

In [ ]:
preds = pd.Series(index=targets.index, data=preds)
preds

# PREDICTION DU MODEL 1 LSTM

In [ ]:
preds = pd.Series(index=targets.index, data=preds)
preds.plot(figsize=(10,8) , c='g')

## VRAIE DONNE DU TEST

In [ ]:
test_SPX.plot(figsize=(10,8), c='r')

## COMPARAISON DES VRAIE DONNEE AVEC LES PREDICTIONS

In [ ]:
line_plot(targets, preds, 'actual', 'prediction', lw=3)

In [ ]:
line_plot(test_SPX, preds, 'actual', 'prediction', lw=3)

## PREDICTION DU MODEL SUR LES DONNEES DE TEST

In [ ]:
train_SPX = SPX.loc[0:1110,'Close']
test_SPX = preds

def line_plot(line1, line2,line3 ,label1=None, label2=None, label3=None,title='', lw=2):
    fig, ax = plt.subplots(1, figsize=(14, 9))
    ax.plot(line1, label=label1, linewidth=lw)
    ax.plot(line2, label=label2, linewidth=lw, c='orange')
    #ax.plot(line2, label=label2, linewidth=lw, c='o')
    ax.plot(line3, label=label3, linewidth=lw, c='g')
    ax.set_ylabel('SPX/USDT', fontsize=14)
    ax.set_title(title, fontsize=16)
    ax.legend(loc='best', fontsize=16);

In [ ]:
line_plot(train_SPX, targets,preds, 'training','test', 'Prediction du modele', title="DONNEE D'ENTRAINEMNT ET DE TEST")

## MODEL 2 Recurrent Neural network AVEC KERAS

In [ ]:
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,GRU
from sklearn.preprocessing import MinMaxScaler,RobustScaler,StandardScaler
from sklearn.metrics import mean_squared_error
from pandas import Series
import keras
from keras import optimizers
from keras.layers import Activation, Dense,Dropout

In [ ]:
df = SPX.set_index('Date')
data=df.copy()

In [ ]:
data.head()

In [ ]:
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return Series(diff)

def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        #takes 
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i+look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

In [ ]:
look_back=7
numpy.random.seed(0)

# load the dataset
dataframe = data
dataset = dataframe.values
dataset = dataset.astype('float64').reshape(-1, 1)
# normalize the dataset
scaler = MinMaxScaler()
#scaler=RobustScaler()
dataset = scaler.fit_transform(dataset)
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
# reshape into X=t and Y=t+1
#look_back = 7
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
# create and fit the LSTM network


## CREATE MODEL

In [ ]:
model = Sequential()

model.add(LSTM(256, return_sequences=True,input_shape=(1, look_back)))
model.add(LSTM(256, return_sequences=True,input_shape=(1, look_back)))
#model.add(LSTM(256, return_sequences=True,input_shape=(1, look_back)))
model.add(LSTM(256))
model.add(Dense(1))

In [ ]:
predictions

## COMPILE MODEL

In [ ]:
#keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False, clipnorm=1)

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=50, verbose=1,shuffle=False,batch_size=50)
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))
predictions = numpy.empty_like(dataset)
predictions[:, :] = numpy.nan
predictions[look_back:len(trainPredict)+look_back, :] = trainPredict
predictions[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
#data=pd.DataFrame(numpy.concatenate((trainPredict[0:len(trainPredict)-look_back-1],testPredict[0:len(testPredict)-look_back-1])),columns=["predicted"])
#print('one',data.count())
#print('two',dataframe.count())
predictionsDF=pd.DataFrame(predictions,columns=["predicted"],index=dataframe.index)
ans=pd.concat([dataframe,predictionsDF],axis=1)
print( ans,[look_back,trainScore,testScore])

In [ ]:
line_plot(testX, testPredict, 'actual', 'prediction', lw=3)

In [ ]:
line_plot(SPX['Close'], predictions, 'actual', 'prediction', lw=3)

In [ ]:
train_SPX = SPX.loc[0:1110,'Close']
test_SPX = preds

def line_plot(line1, line2 ,label1=None, label2=None,title='', lw=2):
    fig, ax = plt.subplots(1, figsize=(14, 9))
    ax.plot(line1, label=label1, linewidth=lw, c='r')
    ax.plot(line2, label=label2, linewidth=lw, c='g')
    ax.set_ylabel('SPX/USDT', fontsize=14)
    ax.set_title(title, fontsize=16)
    ax.legend(loc='best', fontsize=16);


In [ ]:
line_plot(trainPredict, testPredict, 'training', 'Prediction du modele', title="DONNEE D'ENTRAINEMNT ET DE TEST")

In [ ]:
testX_ = scaler.inverse_transform(testX)

In [ ]:
testX_ = testX
testX_.shape

In [ ]:
testX_ = testX_.reshape(-1,)
testX_.shape

In [ ]:
testX_

In [ ]:
testX_ = scaler.inverse_transform(testX_)

In [ ]:
testX_

In [ ]:
line_plot(testX_, testPredict, 'training', 'Prediction du modele', title="DONNEE D'ENTRAINEMNT ET DE TEST")


In [ ]:
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [ ]:
trainPredict.plot(figsize=(10,8), c='r')